# 27. 用列表推导取代map与filter
list,dict,set都可以用列表推导，好处是简洁易懂


In [1]:
a = [1,2,3,4,5,6,7,8,9,10]
a_square = [i**2 for i in a if i%2==0]
print(a_square)

[4, 16, 36, 64, 100]


In [2]:
a_dict_square={i:i**2 for i in a if i%2==0}
print(a_dict_square)

{2: 4, 4: 16, 6: 36, 8: 64, 10: 100}


In [4]:
a_set_square={i**2 for i in a if i%2==0}
print(a_set_square)

{64, 100, 4, 36, 16}


不过要注意：
推导的时候可以使用多层循环，每层循环可以带有多个条件。
控制推导逻辑的子表达式不要超过两个，否则代码很难读懂。

把矩阵（一种二维列表，它的每个元素本身也是列表）转化成普通的一维列表，那么可以在推导时，使用两条for子表达式。这些子表达式会按照从左到右的顺序解读

In [11]:
matrix=[[1,2,3],[4,5,6],[7,8,9]]
flat = [x**2 for row in matrix if len(row) >=0 for x in row ]
print(flat)
new_matrix = [[x**2 for x in row if x %2 ==0] for row in matrix]
print(new_matrix)

[1, 4, 9, 16, 25, 36, 49, 64, 81]
[[4], [16, 36], [64]]


# 29. 用复制表达式消除推导中的重复代码


In [16]:
b = [tmp for i in a if (tmp:= i**2)!=81]
print(b,tmp)  # 新版本里tmp还是会泄漏出去这个和for循环一样

for count in a:
    pass
print(count)

[1, 4, 9, 16, 25, 36, 49, 64, 100] 100
10


如果推导逻辑不带条件，而表示新值的那一部分又使用了:=操作符，那么操作符左边的变量就会泄漏到包含这条推导语句的那个作用域里。如果推导逻辑不带条件，而表示新值的那一部分又使用了:=操作符，那么操作符左边的变量就会泄漏到包含这条推导语句的那个作用域里

# 30. 不让函数直接返回列表，而是逐个生成列表里的值


In [21]:
def index_words_iter(text):
    if text:
        yield 0
    for index,letter in enumerate(text):
        if letter == ' ':
            yield index+1
test_str='a world is used here'
it = index_words_iter(test_str)
print(f'{it},{next(it)}')
#list(it)  迭代器只能产生一次结果。假如迭代器或生成器已经抛出过StopIteration异常，
# 继续用它来构造列表或是像normalize那样对它做for循环，那它不会给出任何结果

for i in it:
    print(f'it index {i}')

<generator object index_words_iter at 0x7f732838aeb0>,0
it index 2
it index 8
it index 11
it index 16


# 31 谨慎迭代函数收到的参数，注意重复迭代时的处理办法

In [26]:
def normalize(numbers):
    total = sum(numbers)
    return [100*value/total for value in numbers ]

test_n = [1,2,3,5,7,8,9]
ret = normalize(test_n)
assert sum(ret) == 100

def genrate_n():
    for i in range(10):
        yield i

ret = normalize(genrate_n())
print(ret) # 这里ret返回值是空，因为在normalize里，sum已经使用过传入的迭代器了，同一个迭代器不能用两次

[]


In [30]:
# 解决方法1,但是这种方法要改动原函数，不优雅
def normalize1(get_iter_func):
    total = sum(get_iter_func())
    return [100*value/total for value in get_iter_func()]

ret = normalize1(lambda:genrate_n())
assert sum(ret) == 100

In [32]:
# 解决方法2，用一种实现迭代器协议的容器

class TestIter:
    def __init__(self,range_len):
        self.length = range_len
    def __iter__(self):
        for i in range(self.length):
            yield i

ret = normalize(TestIter(10))
assert sum(ret)==100

默认的list,dict,set其实都是这种容器，怎样判断一个变量是不是这种实现迭代器协议的容器？

In [37]:
# 方法1 普通的迭代器传给内置的iter函数，那么函数会把迭代器本身返回给调用者。反之，如果传来的是容器类型

def judge_iter(tmp):
    if iter(tmp) is tmp:
        raise TypeError('不支持重复迭代')
try:
    it = genrate_n()
    judge_iter(it)
except TypeError:
    print('不支持重复迭代')

judge_iter(list('abc')) # 默认的list是支持重复迭代的容器

不支持重复迭代


In [38]:
# 方法2 collections.abc内置模块里定义了名为Iterator的类，它用在isinstance函数中，可以判断自己收到的参数是不是这种实例。如果是，就抛出异常
from collections.abc import Iterator

tmp = genrate_n()
if isinstance(tmp,Iterator):
    print('不支持重复迭代')

不支持重复迭代


# 32 用生成表达式改写数据量较大的列表推导
